# Object Detection with TensorFlow Lite Model Maker

## Prerequisites


In [2]:
!pip install -q --use-deprecated=legacy-resolver tflite-model-maker
!pip install -q pycocotools

     |████████████████████████████████| 616 kB 12.3 MB/s 
     |████████████████████████████████| 120 kB 48.8 MB/s 
     |████████████████████████████████| 3.4 MB 49.3 MB/s 
     |████████████████████████████████| 77 kB 5.3 MB/s 
     |████████████████████████████████| 1.1 MB 36.9 MB/s 
     |████████████████████████████████| 234 kB 42.4 MB/s 
     |████████████████████████████████| 6.4 MB 2.7 MB/s 
     |████████████████████████████████| 87 kB 5.9 MB/s 
     |████████████████████████████████| 596 kB 42.7 MB/s 
     |████████████████████████████████| 1.2 MB 36.0 MB/s 
     |████████████████████████████████| 840 kB 35.4 MB/s 
     |████████████████████████████████| 1.1 MB 27.8 MB/s 
     |████████████████████████████████| 25.3 MB 4.1 MB/s 
     |████████████████████████████████| 211 kB 32.6 MB/s 
     |████████████████████████████████| 462 kB 10.3 MB/s 
     |████████████████████████████████| 47.7 MB 96 kB/s 
     |████████████████████████████████| 352 kB 33.7 MB/s 
     |██████████████

Import the required packages.

In [3]:
import numpy as np
import os

from tflite_model_maker.config import QuantizationConfig
from tflite_model_maker.config import ExportFormat
from tflite_model_maker import model_spec
from tflite_model_maker import object_detector

import tensorflow as tf
assert tf.__version__.startswith('2')

tf.get_logger().setLevel('ERROR')
from absl import logging
logging.set_verbosity(logging.ERROR)

## Quickstart

There are six steps to training an object detection model:

**Step 1. Choose an object detection model archiecture.**

In [4]:
spec = model_spec.get('efficientdet_lite0')

**Step 2. Load the dataset.**

In [5]:
train_data, validation_data, test_data = object_detector.DataLoader.from_csv('gs://cloud-ml-data/img/openimage/csv/salads_ml_use.csv')

**Step 3. Train the TensorFlow model with the training data.**

In [6]:
model = object_detector.create(train_data, model_spec=spec, batch_size=8, train_whole_model=True, validation_data=validation_data)

Epoch 1/50
21/21 [==============================] - 93s 2s/step - det_loss: 1.7696 - cls_loss: 1.1191 - box_loss: 0.0130 - reg_l2_loss: 0.0635 - loss: 1.8331 - learning_rate: 0.0090 - gradient_norm: 0.8542 - val_det_loss: 1.6091 - val_cls_loss: 1.0512 - val_box_loss: 0.0112 - val_reg_l2_loss: 0.0635 - val_loss: 1.6726
Epoch 2/50
21/21 [==============================] - 32s 2s/step - det_loss: 1.5647 - cls_loss: 1.0197 - box_loss: 0.0109 - reg_l2_loss: 0.0635 - loss: 1.6282 - learning_rate: 0.0100 - gradient_norm: 1.2811 - val_det_loss: 1.4324 - val_cls_loss: 0.9174 - val_box_loss: 0.0103 - val_reg_l2_loss: 0.0635 - val_loss: 1.4959
Epoch 3/50
21/21 [==============================] - 36s 2s/step - det_loss: 1.3496 - cls_loss: 0.8704 - box_loss: 0.0096 - reg_l2_loss: 0.0635 - loss: 1.4132 - learning_rate: 0.0099 - gradient_norm: 1.6779 - val_det_loss: 1.2514 - val_cls_loss: 0.7673 - val_box_loss: 0.0097 - val_reg_l2_loss: 0.0635 - val_loss: 1.3149
Epoch 4/50
21/21 [======================

**Step 4. Evaluate the model with the test data.**

In [7]:
model.evaluate(test_data)

1/1 [==============================] - 12s 12s/step



{'AP': 0.21259458,
 'AP50': 0.34831882,
 'AP75': 0.21688399,
 'AP_/Baked Goods': 0.057555556,
 'AP_/Cheese': 0.16166192,
 'AP_/Salad': 0.58795625,
 'AP_/Seafood': 0.015181394,
 'AP_/Tomato': 0.24061775,
 'APl': 0.20892134,
 'APm': 0.59559995,
 'APs': -1.0,
 'ARl': 0.34869495,
 'ARm': 0.65833336,
 'ARmax1': 0.15383689,
 'ARmax10': 0.31281304,
 'ARmax100': 0.35153744,
 'ARs': -1.0}

**Step 5.  Export as a TensorFlow Lite model.**

In [8]:
model.export(export_dir='.')

**Step 6.  Evaluate the TensorFlow Lite model.**

In [9]:
model.evaluate_tflite('model.tflite', test_data)

25/25 [==============================] - 72s 3s/step



{'AP': 0.19997387,
 'AP50': 0.3317277,
 'AP75': 0.21221836,
 'AP_/Baked Goods': 0.0,
 'AP_/Cheese': 0.18583816,
 'AP_/Salad': 0.56794775,
 'AP_/Seafood': 0.0,
 'AP_/Tomato': 0.24608341,
 'APl': 0.19614685,
 'APm': 0.5128933,
 'APs': -1.0,
 'ARl': 0.27709496,
 'ARm': 0.56666666,
 'ARmax1': 0.1279356,
 'ARmax10': 0.2566366,
 'ARmax100': 0.280508,
 'ARs': -1.0}